<a href="https://colab.research.google.com/github/elichen/aoc2018/blob/main/Day_15_Beverage_Bandits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def parse_map(map_2d):
    map_list = [list(row) for row in map_2d]
    return np.array(map_list)

In [ ]:
import numpy as np
import heapq

def dijkstra(grid, start, end):
    directions = [(-1, 0), (0, -1), (0, 1), (1, 0)]  # Reading order: up, left, right, down
    open_set = []
    heapq.heappush(open_set, (0, start))

    came_from = {}
    g_score = {start: 0}

    while open_set:
        current_distance, current = heapq.heappop(open_set)

        if current == end:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for d in directions:
            neighbor = (current[0] + d[0], current[1] + d[1])
            if 0 <= neighbor[0] < grid.shape[0] and 0 <= neighbor[1] < grid.shape[1] and grid[neighbor] == '.':
                tentative_g_score = g_score[current] + 1
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    heapq.heappush(open_set, (g_score[neighbor], neighbor))

    return None

In [ ]:
# Function to pretty print the grid
def pretty_print_grid(grid):
    for row in grid:
        print(''.join(row))

In [ ]:
from itertools import product
import numpy as np
import heapq

def is_adjacent(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1]) == 1

def simulate_round(grid, unit_hp=None):
    units = 'GE'  # Unit types: Goblins and Elves
    attack_power = {'G': 3, 'E': 3}  # Attack power for Goblins and Elves

    if unit_hp is None:
        unit_hp = {}

    if not unit_hp:
        for y, x in product(range(grid.shape[0]), range(grid.shape[1])):
            if grid[y, x] in units:
                unit_hp[(y, x)] = 200  # Each unit starts with 200 hit points

    def get_targets(unit_type, grid, unit_pos):
        enemy_type = 'G' if unit_type == 'E' else 'E'
        return [pos for pos, type_ in unit_pos.items() if type_ == enemy_type]

    def get_reachable_squares(unit_pos, targets, grid):
        squares = []
        for target in targets:
            for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ny, nx = target[0] + dy, target[1] + dx
                if 0 <= ny < grid.shape[0] and 0 <= nx < grid.shape[1] and grid[ny, nx] == '.':
                    squares.append((ny, nx))
        return squares

    def move_unit(unit_pos, target_squares, grid):
        paths = [(dijkstra(grid, unit_pos, sq), sq) for sq in target_squares]
        paths = [(p, s) for p, s in paths if p is not None]
        if not paths:
            return unit_pos
        shortest_path = min(paths, key=lambda x: (len(x[0]), x[1]))[0]
        return shortest_path[1]

    def get_adjacent_targets(unit_type, pos, unit_pos):
      enemy_type = 'G' if unit_type == 'E' else 'E'
      return [p for p in unit_pos if is_adjacent(pos, p) and grid[p] == enemy_type]

    def attack(unit, targets, unit_hp, unit_pos):
        if not targets:
            return
        target = min(targets, key=lambda x: (unit_hp[x], x))
        unit_hp[target] -= attack_power[unit]
        if unit_hp[target] <= 0:
            grid[target] = '.'
            del unit_hp[target]
            del unit_pos[target]

    unit_pos = {(y, x): grid[y, x] for y, x in product(range(grid.shape[0]), range(grid.shape[1])) if grid[y, x] in units}

    for pos, unit in sorted(unit_pos.items(), key=lambda x: x[0]):
        if grid[pos] != unit or pos not in unit_hp:
            continue

        targets = get_targets(unit, grid, unit_pos)
        if not targets:
            return grid, unit_hp, True  # Combat ends

        target_squares = get_reachable_squares(pos, targets, grid)
        if not any(is_adjacent(pos, t) for t in targets):
            if target_squares:
                new_position = move_unit(pos, target_squares, grid)
                grid[pos], grid[new_position] = '.', unit
                unit_pos[new_position] = unit_pos.pop(pos)
                unit_hp[new_position] = unit_hp.pop(pos)
                pos = new_position

        adjacent_targets = get_adjacent_targets(unit, pos, unit_pos)
        attack(unit, adjacent_targets, unit_hp, unit_pos)

    return grid, unit_hp, False

In [ ]:
data1 = """#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######""".split('\n')
data2 = """#######
#G..#E#
#E#E.E#
#G.##.#
#...#E#
#...E.#
#######""".split('\n')
data3 = """#######
#E..EG#
#.#G.E#
#E.##E#
#G..#.#
#..E#.#
#######""".split('\n')
data4 = """#######
#E.G#.#
#.#G..#
#G.#.G#
#G..#.#
#...E.#
#######""".split('\n')
data5 = """#######
#.E...#
#.#..G#
#.###.#
#E#G#G#
#...#G#
#######""".split('\n')
data6 = """#########
#G......#
#.E.#...#
#..##..G#
#...##..#
#...#...#
#.G...G.#
#.....G.#
#########""".split('\n')

In [ ]:
data = """################################
#########...####################
#########...###########.########
#########G..##########....######
##########..###########...######
#########G...##########...######
#########..G.###########..######
########...#.##########..#######
#######G#..###E######....#######
#######G.....#.######....#######
######...G......##E......#######
####...##.#..G..G.........######
###..........G#####.......####.#
####........G#######...........#
####..G.....#########......#...#
###.........#########........###
##.....G.G..#########......#####
#...G.......#########.........##
#.G.........#########.E.##...###
##.....G.....#######....G#.E...#
##............#####...E.......##
#.G...........E.......#E...##.##
#....G........###########.....##
#......##...#.##################
#.#.........E..##.##############
#.#.......G.......##############
#.###........E....##############
#.####.....###....##############
#.#####......E..################
#######..........###############
#########..####.################
################################""".split('\n')

In [ ]:
grid = parse_map(data)
hps = None
round = 0
while True:
  grid, hps, end = simulate_round(grid, hps)
  if end:
    break
  round += 1
  # pretty_print_grid(grid)
  # print(round, hps, full)
pretty_print_grid(grid)
round, sum(hps.values()), round * sum(hps.values())

################################
#########...####################
#########...###########.########
#########G..##########....######
##########..###########...######
#########G...##########...######
#########..G.###########..######
########...#.##########..#######
#######G#..###E######....#######
#######G.....#.######....#######
######...G......##E......#######
####...##.#..G..G.........######
###..........G#####.......####.#
####........G#######...........#
####..G.....#########......#...#
###.........#########........###
##.....G.G..#########......#####
#...G.......#########.........##
#.G.........#########.E.##...###
##.....G.....#######....G#.E...#
##............#####...E.......##
#.G...........E.......#E...##.##
#....G........###########.....##
#......##...#.##################
#.#.........E..##.##############
#.#.......G.......##############
#.###........E....##############
#.####.....###....##############
#.#####......E..################
#######..........###############
#########.

(149, 2326, 346574)

In [ ]:
from itertools import product
import numpy as np
import heapq

def is_adjacent(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1]) == 1

def simulate_round2(grid, unit_hp=None, elf=3):
    units = 'GE'  # Unit types: Goblins and Elves
    attack_power = {'G': 3, 'E': elf}  # Attack power for Goblins and Elves

    if unit_hp is None:
        unit_hp = {}

    if not unit_hp:
        for y, x in product(range(grid.shape[0]), range(grid.shape[1])):
            if grid[y, x] in units:
                unit_hp[(y, x)] = 200  # Each unit starts with 200 hit points

    def get_targets(unit_type, grid, unit_pos):
        enemy_type = 'G' if unit_type == 'E' else 'E'
        return [pos for pos, type_ in unit_pos.items() if type_ == enemy_type]

    def get_reachable_squares(unit_pos, targets, grid):
        squares = []
        for target in targets:
            for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ny, nx = target[0] + dy, target[1] + dx
                if 0 <= ny < grid.shape[0] and 0 <= nx < grid.shape[1] and grid[ny, nx] == '.':
                    squares.append((ny, nx))
        return squares

    def move_unit(unit_pos, target_squares, grid):
        paths = [(dijkstra(grid, unit_pos, sq), sq) for sq in target_squares]
        paths = [(p, s) for p, s in paths if p is not None]
        if not paths:
            return unit_pos
        shortest_path = min(paths, key=lambda x: (len(x[0]), x[1]))[0]
        return shortest_path[1]

    def get_adjacent_targets(unit_type, pos, unit_pos):
      enemy_type = 'G' if unit_type == 'E' else 'E'
      return [p for p in unit_pos if is_adjacent(pos, p) and grid[p] == enemy_type]

    def attack(unit, targets, unit_hp, unit_pos):
        if not targets:
            return
        target = min(targets, key=lambda x: (unit_hp[x], x))
        unit_hp[target] -= attack_power[unit]
        if unit_hp[target] <= 0:
            grid[target] = '.'
            del unit_hp[target]
            del unit_pos[target]

    unit_pos = {(y, x): grid[y, x] for y, x in product(range(grid.shape[0]), range(grid.shape[1])) if grid[y, x] in units}

    for pos, unit in sorted(unit_pos.items(), key=lambda x: x[0]):
        if grid[pos] != unit or pos not in unit_hp:
            continue

        targets = get_targets(unit, grid, unit_pos)
        if not targets:
            return grid, unit_hp, True  # Combat ends

        target_squares = get_reachable_squares(pos, targets, grid)
        if not any(is_adjacent(pos, t) for t in targets):
            if target_squares:
                new_position = move_unit(pos, target_squares, grid)
                grid[pos], grid[new_position] = '.', unit
                unit_pos[new_position] = unit_pos.pop(pos)
                unit_hp[new_position] = unit_hp.pop(pos)
                pos = new_position

        adjacent_targets = get_adjacent_targets(unit, pos, unit_pos)
        attack(unit, adjacent_targets, unit_hp, unit_pos)

    return grid, unit_hp, False

In [ ]:
pretty_print_grid(parse_map(data))
for elf in range(10,100):
  grid = parse_map(data)
  hps = None
  round = 0
  while True:
    grid, hps, end = simulate_round2(grid, hps, elf)
    if end:
      break
    round += 1
    # pretty_print_grid(grid)
    # print(round, hps, full)
  if np.sum(grid == 'E') == np.sum(parse_map(data) == 'E'):
    break
pretty_print_grid(grid)
print(elf, round, sum(hps.values()), round * sum(hps.values()))

################################
#########...####################
#########...###########.########
#########G..##########....######
##########..###########...######
#########G...##########...######
#########..G.###########..######
########...#.##########..#######
#######G#..###E######....#######
#######G.....#.######....#######
######...G......##E......#######
####...##.#..G..G.........######
###..........G#####.......####.#
####........G#######...........#
####..G.....#########......#...#
###.........#########........###
##.....G.G..#########......#####
#...G.......#########.........##
#.G.........#########.E.##...###
##.....G.....#######....G#.E...#
##............#####...E.......##
#.G...........E.......#E...##.##
#....G........###########.....##
#......##...#.##################
#.#.........E..##.##############
#.#.......G.......##############
#.###........E....##############
#.####.....###....##############
#.#####......E..################
#######..........###############
#########.